In [1]:
import re
import numpy as np
import sys
from pathlib import Path

sys.path.append(str(Path("/home/yann/ssd_storage/python/arcprize2025/tests/")))
sys.path.append(str(Path("/home/yann/ssd_storage/python/arcprize2025/sources/")))

from test_dsl_symbolic_executor import TEST_CASES
from core.dsl_symbolic_interpreter import SYMBOL_RULES

In [2]:
ATOMIC_PATTERNS = []
CONDITIONALLY_ATOMIC_PATTERNS_CHECKERS = {}
NON_ATOMIC_PATTERNS = []

for rule_key, rule_definition in SYMBOL_RULES.items():
    sigil = rule_definition.get("sigil")
    pattern = rule_definition.get("pattern")
    nested_commands = rule_definition.get("nested_commands")

    if sigil and not pattern and not nested_commands:
        ATOMIC_PATTERNS.append(re.compile(rf"^{re.escape(sigil)}$"))
        continue

    if not pattern:
        continue

    try:
        compiled_pattern = re.compile(pattern)
    except re.error:
        continue

    if nested_commands is None or nested_commands == {}:
        ATOMIC_PATTERNS.append(compiled_pattern)
    elif rule_key in ["flip_h", "flip_v", "flatten_grid", "extract_bounding_box", "reverse_row"]:
        def make_checker(rule_key_inner):
            def checker(match):
                arg_content = match.group("arg_content") if match.lastindex and match.lastgroup == "arg_content" else None
                return arg_content is None or arg_content.strip() == "⌂"
            return checker
        CONDITIONALLY_ATOMIC_PATTERNS_CHECKERS[compiled_pattern] = make_checker(rule_key)
    else:
        NON_ATOMIC_PATTERNS.append(compiled_pattern)

block_builder_def = SYMBOL_RULES.get("block_grid_builder")
if block_builder_def and "pattern" in block_builder_def:
    try:
        ATOMIC_PATTERNS.append(re.compile(block_builder_def["pattern"]))
    except re.error:
        pass

In [3]:
def is_atomic_rule(rule_str: str) -> bool:
    rule_str = rule_str.strip()

    for non_atomic_pattern in NON_ATOMIC_PATTERNS:
        if non_atomic_pattern.match(rule_str):
            return False

    for atomic_pattern in ATOMIC_PATTERNS:
        if atomic_pattern.match(rule_str):
            return True

    for conditional_pattern, checker in CONDITIONALLY_ATOMIC_PATTERNS_CHECKERS.items():
        match = conditional_pattern.match(rule_str)
        if match and checker(match):
            return True

    return False

In [4]:
atomic_training_data = []

for rule_str, input_grid_np, _ in TEST_CASES:
    if not is_atomic_rule(rule_str):
        continue

    if input_grid_np is not None:
        input_grid_list = input_grid_np.tolist()
    else:
        if "▦(" in rule_str:
             input_grid_list = []
        else:
             continue

    atomic_training_data.append({
        "input_grid": input_grid_list,
        "dsl_rule": rule_str
    })

print(f"Found {len(atomic_training_data)} atomic training examples.")


Found 120 atomic training examples.


In [5]:
import numpy as np
import sys
from pathlib import Path

def execute_dsl_rule_on_grid(input_grid_list: list[list[int]], dsl_rule_str: str) -> np.ndarray:
    
    current_dir = Path.cwd()
    project_root_candidates = [
        current_dir,
        current_dir.parent,
        current_dir.parent.parent
    ]
    
    src_path = None
    for root_candidate in project_root_candidates:
        if (root_candidate / "sources").exists():
            src_path = root_candidate / "sources"
            break
            
    if src_path is None:
        raise FileNotFoundError("Could not find the 'sources' directory. Please ensure your working directory is correctly set relative to 'sources' or manually adjust the 'src_path' variable in this function.")

    if str(src_path) not in sys.path:
        sys.path.append(str(src_path))

    try:
        from core.dsl_symbolic_interpreter import SymbolicRuleParser
        from core.dsl_symbolic_executor import DSLExecutor
    except ImportError as e:
        raise ImportError(f"Failed to import DSL components. Ensure 'core/dsl_symbolic_interpreter.py' and 'core/dsl_symbolic_executor.py' exist in your '{src_path}' directory. Error: {e}")

    input_grid_np = np.array(input_grid_list, dtype=int)
    parser = SymbolicRuleParser()

    try:
        command = parser.parse_rule(dsl_rule_str)
        executor = DSLExecutor(
            root_command=command,
            initial_puzzle_input=input_grid_np,
        )
        result_grid_np = executor.execute_program()
        return result_grid_np
    except Exception as e:
        print(f"Error executing rule '{dsl_rule_str}' on input grid: {e}")
        return np.array([]) 

In [ ]:
import numpy as np
import random

def int_to_roman(num: int) -> str:
    mapping = {
        0: "∅", 1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V',
        6: 'VI', 7: 'VII', 8: 'VIII', 9: 'IX', 10: 'X',
        11: 'XI', 12: 'XII', 13: 'XIII', 14: 'XIV', 15: 'XV',
        16: 'XVI', 17: 'XVII', 18: 'XVIII', 19: 'XIX', 20: 'XX',
        21: 'XXI', 22: 'XXII', 23: 'XXIII', 24: 'XXIV', 25: 'XXV',
        26: 'XXVI', 27: 'XXVII', 28: 'XXVIII', 29: 'XXIX', 30: 'XXX'
    }
    return mapping[num]

        
def roman_to_int(roman_numeral: str) -> int:
    mapping = {
        '∅': 0, # Add zero mapping
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5,
        'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10,
        'XI': 11, 'XII': 12, 'XIII': 13, 'XIV': 14, 'XV': 15,
        'XVI': 16, 'XVII': 17, 'XVIII': 18, 'XIX': 19, 'XX': 20,
        'XXI': 21, 'XXII': 22, 'XXIII': 23, 'XXIV': 24, 'XXV': 25,
        'XXVI': 26, 'XXVII': 27, 'XXVIII': 28, 'XXIX': 29, 'XXX': 30
    }
    return mapping.get(roman_numeral.upper(), None)


def is_rule_compatible_with_grid(dsl_rule_str: str, grid_shape: tuple) -> bool:
    rows, cols = grid_shape

    if rows < 1 or cols < 1:
        return False

    if dsl_rule_str.startswith('⇅('):
        match = re.match(r'⇅\(([IVX]+),([IVX]+)\)', dsl_rule_str)
        if not match:
            print(f"Warning: Malformed ⇅ rule string encountered: {dsl_rule_str}")
            return False
        
        row_str1, row_str2 = match.groups()
        idx1 = roman_to_int(row_str1)
        idx2 = roman_to_int(row_str2)

        if idx1 > 0 and idx2 > 0 and idx1 <= rows and idx2 <= rows:
            return True
        else:
            return False
        
    return True

def generate_single_random_grid(original_grid_shape: tuple, max_dim: int = 30, num_range: int = 9) -> np.ndarray:
    rows_orig, cols_orig = original_grid_shape if original_grid_shape and len(original_grid_shape) == 2 else (0,0)

    if random.random() < 0.7:
        new_rows = random.randint(1, max_dim + 1)
        new_cols = random.randint(1, max_dim + 1)
        new_values = np.random.choice(range(1, num_range + 1), size=new_rows * new_cols, replace=True)
        new_grid = new_values.reshape(new_rows, new_cols)
    else:
        if rows_orig > 0 and cols_orig > 0:
            while True:
                new_values = np.random.choice(range(1, num_range + 1), size=rows_orig * cols_orig, replace=True)
                temp_grid = new_values.reshape(rows_orig, cols_orig)
                if not np.array_equal(temp_grid, np.zeros((rows_orig, cols_orig))) or np.array_equal(temp_grid, np.full((rows_orig, cols_orig), num_range)):
                    new_grid = temp_grid
                    break
            
        else:
            new_grid = np.array([[random.randint(1, num_range)]])
    
    return new_grid

def mutate_flip_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    mutated_items = []
    original_grid = np.array(item['input_grid'], dtype=int)
    original_rule = item['dsl_rule']

    for _ in range(num_variants - 1):
        new_grid = generate_single_random_grid(original_grid.shape, max_dim, num_range)
        mutated_items.append({'input_grid': new_grid.tolist(), 'dsl_rule': original_rule})

    return mutated_items

def mutate_swap_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9, is_row_swap: bool = True) -> list[dict]:
    mutated_items = []
    original_rule = item['dsl_rule']
    pattern = r'⇅\((?P<idx1>[IVX]+),(?P<idx2>[IVX]+)\)' if is_row_swap else r'⇄\((?P<idx1>[IVX]+),(?P<idx2>[IVX]+)\)'
    sigil = '⇅' if is_row_swap else '⇄'
    
    match = re.match(pattern, original_rule)
    if not match:
        print(f"Warning: Could not parse swap rule: {original_rule}")
        return []
    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 100
    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        candidate_grid_np = generate_single_random_grid(np.array(item['input_grid']).shape, max_dim, num_range)
        dim_size = candidate_grid_np.shape[0] if is_row_swap else candidate_grid_np.shape[1]
        if dim_size >= 2:
            new_idx1_idx = random.randint(1, dim_size)
            new_idx2_idx = random.randint(1, dim_size)
            while new_idx2_idx == new_idx1_idx:
                new_idx2_idx = random.randint(1, dim_size)
            new_idx1_name = int_to_roman(new_idx1_idx)
            new_idx2_name = int_to_roman(new_idx2_idx)
            new_rule = f"{sigil}({new_idx1_name},{new_idx2_name})"
            mutated_items.append({
                'input_grid': candidate_grid_np.tolist(),
                'dsl_rule': new_rule
            })
        attempts += 1
    if len(mutated_items) < target_count:
        print(f"Warning: mutate_swap_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")
    return mutated_items

def mutate_swap_row_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    return mutate_swap_rule(item, num_variants, max_dim, num_range, is_row_swap=True)

def mutate_swap_col_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    return mutate_swap_rule(item, num_variants, max_dim, num_range, is_row_swap=False)

def mutate_swap_value_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    """
    Mutates a value swap rule (⇒(A, B)) by generating new grids and updating the values in the rule.
    Ensures the 'from' value (A) exists in the new grid.
    """
    mutated_items = []
    original_rule = item['dsl_rule']
    match = re.match(r'⇒\((?P<from_val>[^,]*),\s*(?P<to_val>[^)]*)\)', original_rule)
    if not match:
        print(f"Warning: Could not parse swap value rule: {original_rule}")
        return []

    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 100

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        # 1. Generate a new candidate grid
        candidate_grid_np = generate_single_random_grid(np.array(item['input_grid']).shape, max_dim, num_range)
        
        # 2. Determine new values for the rule (0 to num_range)
        new_from_val_int = random.randint(0, num_range)
        new_to_val_int = random.randint(0, num_range)
        
        # Convert integers to DSL symbols using the updated int_to_roman
        new_from_val_sym = int_to_roman(new_from_val_int)
        new_to_val_sym = int_to_roman(new_to_val_int)

        # 3. Create the new rule string
        new_rule = f"⇒({new_from_val_sym}, {new_to_val_sym})"

        # 4. Ensure the 'from' value exists in the grid, or inject it
        if new_from_val_int in candidate_grid_np:
            # Value already exists, grid is fine
            final_grid = candidate_grid_np
        else:
            # Need to inject the 'from' value to make the rule meaningful
            # Modify one random element to be the 'from' value
            if candidate_grid_np.size > 0:
                flat_index = random.randint(0, candidate_grid_np.size - 1)
                final_grid = candidate_grid_np.copy()
                final_grid.flat[flat_index] = new_from_val_int
            else:
                final_grid = candidate_grid_np # Keep empty grid if size is 0

        mutated_items.append({
            'input_grid': final_grid.tolist(),
            'dsl_rule': new_rule
        })
        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_swap_value_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")

    return mutated_items

def mutate_extract_value_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    """
    Mutates an extract value rule (⊡(R,C)) by generating new grids and updating the row/column indices.
    Ensures the new indices are within the bounds of the new grid.
    """
    mutated_items = []
    original_rule = item['dsl_rule']
    match = re.match(r'⊡\((?P<row>[IVX∅]+),(?P<col>[IVX∅]+)\)', original_rule)
    if not match:
        print(f"Warning: Could not parse extract value rule: {original_rule}")
        return []

    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 100

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        # 1. Generate a new candidate grid
        candidate_grid_np = generate_single_random_grid(np.array(item['input_grid']).shape, max_dim, num_range)
        
        rows, cols = candidate_grid_np.shape
        
        # Need at least a 1x1 grid to extract a value
        if rows >= 1 and cols >= 1:
            # 2. Select valid 1-based indices for the new grid
            new_row_idx = random.randint(1, rows)
            new_col_idx = random.randint(1, cols)

            # 3. Convert indices to DSL symbols
            new_row_sym = int_to_roman(new_row_idx)
            new_col_sym = int_to_roman(new_col_idx)

            # 4. Create the new rule string
            new_rule = f"⊡({new_row_sym},{new_col_sym})"

            mutated_items.append({
                'input_grid': candidate_grid_np.tolist(),
                'dsl_rule': new_rule
            })
            
        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_extract_value_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")

    return mutated_items

def mutate_identity_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    """
    Mutates an identity rule (⌂) by generating new random grids.
    The rule string '⌂' remains unchanged.
    """
    mutated_items = []
    original_rule = item['dsl_rule'] # This should be '⌂'
    
    for _ in range(num_variants - 1):
        # 1. Generate a new candidate grid
        candidate_grid_np = generate_single_random_grid(np.array(item['input_grid']).shape, max_dim, num_range)
        
        # 2. Append the new grid with the unchanged rule
        mutated_items.append({
            'input_grid': candidate_grid_np.tolist(),
            'dsl_rule': original_rule # Remains '⌂'
        })
        
    return mutated_items

def mutate_flood_fill_rule(item: dict, num_variants: int = 3, max_dim: int = 15, num_range: int = 9) -> list[dict]:
    """
    Mutates a flood fill rule (<tool_call>(V)) by generating new grids with structures
    suitable for the flood-fill background operation and updating the value argument.
    """
    mutated_items = []
    original_rule = item['dsl_rule']
    # Correctly parse the rule to get the original value symbol
    match = re.match(r'⏚\((?P<value>[^)]*)\)', original_rule) # Ensure '</tool_call>' is correctly escaped if needed by your environment
    if not match:
        print(f"Warning: Could not parse flood fill rule: {original_rule}")
        # Fallback: Use the simpler mutate_for_indexed_rule logic if parsing fails,
        # though it won't change the rule sigil part.
        return None

    original_value_sym = match.group("value").strip()
    original_value_int = roman_to_int(original_value_sym)
    if original_value_int == -1:
        # If parsing the original value symbol fails, default to 0 or handle error
        original_value_int = 0

    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 150 # Increased attempts

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        # --- Grid Generation Strategy for `<tool_call>` ---
        # 1. Decide grid dimensions
        rows = random.randint(3, max_dim)
        cols = random.randint(3, max_dim)
        grid_shape = (rows, cols)

        # 2. Decide on the new target value for the rule
        new_value_int = random.randint(0, num_range)
        new_value_sym = int_to_roman(new_value_int)

        # 3. Create the new rule string
        new_rule = f"</tool_call>({new_value_sym})"

        # 4. Generate a base grid, preferably filled with a different value
        background_value = (new_value_int + random.randint(1, num_range)) % (num_range + 1)
        # Ensure background is different from the new target value
        while background_value == new_value_int and num_range > 0:
            background_value = (background_value + 1) % (num_range + 1)

        candidate_grid_np = np.full(grid_shape, background_value, dtype=int)

        # 5. Strategically place the new_value_int to create interesting patterns for `<tool_call>`
        # Aim for patterns where new_value_int touches the border (background) or forms enclosures.
        pattern_type = random.choices(
            ['border_line', 'border_box', 'enclosed_shape', 'mixed_struct'],
            weights=[30, 30, 25, 15], k=1
        )[0]

        if pattern_type == 'border_line':
            # Place a line of the target value along one border
            side = random.choice(['top', 'bottom', 'left', 'right'])
            if side in ['top', 'bottom']:
                row = 0 if side == 'top' else rows - 1
                candidate_grid_np[row, :] = new_value_int
            else: # left, right
                col = 0 if side == 'left' else cols - 1
                candidate_grid_np[:, col] = new_value_int

        elif pattern_type == 'border_box':
            # Create a solid or hollow box of the target value touching at least one border
            # Simplified: Solid box in a corner
            box_rows = random.randint(1, max(1, rows // 2))
            box_cols = random.randint(1, max(1, cols // 2))
            corner = random.choice(['tl', 'tr', 'bl', 'br'])
            r_start, c_start = 0, 0
            if corner == 'tr': c_start = cols - box_cols
            elif corner == 'bl': r_start = rows - box_rows
            elif corner == 'br': r_start, c_start = rows - box_rows, cols - box_cols
            candidate_grid_np[r_start:r_start+box_rows, c_start:c_start+box_cols] = new_value_int

        elif pattern_type == 'enclosed_shape':
            # Create a simple enclosed shape (like a hollow square) if grid is large enough
            if rows >= 4 and cols >= 4:
                offset = random.randint(1, min(2, rows // 3, cols // 3)) # Keep offset small
                r_start, r_end = offset, rows - offset
                c_start, c_end = offset, cols - offset
                if r_end > r_start and c_end > c_start:
                    # Hollow square
                    candidate_grid_np[r_start, c_start:c_end] = new_value_int # top
                    candidate_grid_np[r_end-1, c_start:c_end] = new_value_int # bottom
                    candidate_grid_np[r_start:r_end, c_start] = new_value_int # left
                    candidate_grid_np[r_start:r_end, c_end-1] = new_value_int # right
                    # Optionally, place a different value inside to make it more interesting
                    # though `new_value_int` on the border is key for `</tool_call>`.
                    # inner_value = (new_value_int + 1) % (num_range + 1)
                    # candidate_grid_np[r_start+1:r_end-1, c_start+1:c_end-1] = inner_value

        elif pattern_type == 'mixed_struct':
            # Combine border placement with some internal scattering
            # Place border line(s)
            for side in random.choices(['top', 'bottom', 'left', 'right'], k=random.randint(1, 3)):
                 if side in ['top', 'bottom']:
                    row = 0 if side == 'top' else rows - 1
                    # Place a segment, not necessarily the full line
                    c1 = random.randint(0, cols - 2)
                    c2 = random.randint(c1 + 1, cols)
                    candidate_grid_np[row, c1:c2] = new_value_int
                 else: # left, right
                    col = 0 if side == 'left' else cols - 1
                    r1 = random.randint(0, rows - 2)
                    r2 = random.randint(r1 + 1, rows)
                    candidate_grid_np[r1:r2, col] = new_value_int
            # Add a few scattered points
            num_points = random.randint(1, max(1, rows * cols // 10))
            for _ in range(num_points):
                r, c = random.randint(0, rows-1), random.randint(0, cols-1)
                candidate_grid_np[r, c] = new_value_int


        # 6. Add a small amount of other noise values, avoiding the new_value_int
        noise_values = [v for v in range(num_range + 1) if v != new_value_int]
        if noise_values:
            num_noise = random.randint(0, max(0, (rows * cols) // 20)) # Very sparse noise
            for _ in range(num_noise):
                r, c = random.randint(0, rows-1), random.randint(0, cols-1)
                # Avoid overwriting the strategically placed new_value_int
                if candidate_grid_np[r, c] != new_value_int:
                    candidate_grid_np[r, c] = random.choice(noise_values)

        # --- Execution and Validation ---
        # Execute the NEW rule on the NEW grid
        try:
            output_grid_np = execute_dsl_rule_on_grid(candidate_grid_np.tolist(), new_rule)
            # Check if execution was successful (didn't return empty array [])
            # and that the output is non-trivial (not all zeros, unless input was all target value)
            if output_grid_np.size > 0:
                # Optional: Add a check to ensure output is not just all 0s if input wasn't trivial
                # This is a basic check, `<tool_call>` can produce all 0s legitimately.
                # A more robust check would involve understanding the `<tool_call>` logic better.
                mutated_items.append({
                    'input_grid': candidate_grid_np.tolist(),
                    'dsl_rule': new_rule, # Crucially, use the NEW rule
                    'output_grid': output_grid_np.tolist()
                })
            # If execution failed or produced empty output, the item is not appended.
            # The loop will continue to try and generate a valid one.
        except Exception as e:
            # If execution raises an exception, don't add the item.
            # print(f"Execution error for rule '{new_rule}': {e}") # Uncomment for debugging
            pass

        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_flood_fill_rule could only generate {len(mutated_items)} valid mutations for rule '{original_rule}' after {attempts} attempts.")

    return mutated_items

In [36]:
# 1. Sort the list by the first character of the 'dsl_rule'
atomic_training_data.sort(key=lambda item: item['dsl_rule'][0])

# 2. Group the sorted list by the first character of the 'dsl_rule'
from itertools import groupby
grouped_data = groupby(atomic_training_data, key=lambda item: item['dsl_rule'][0])
groups = {key: list(group) for key, group in grouped_data}

In [37]:
outputed_dataset = []

In [38]:
mutation_functions_map = {
    # '↢': (lambda item, num_variants, **kwargs: [], 2)
    # '↔': [mutate_flip_rule, 2],
    # '↕': [mutate_flip_rule, 2],
    # '⇅': [mutate_swap_row_rule, 2],
    # '⇄': [mutate_swap_col_rule, 2],
    # '⇒': [mutate_swap_value_rule, 2],
    # '⊡': [mutate_extract_value_rule, 3],
    # '⌂': [mutate_identity_rule, 3] ,
    '⏚': [mutate_flood_fill_rule, 2] 
}

for dsl_symbol, initial_rules_list in groups.items():
    mutate_function , num_variants_per_rule = mutation_functions_map.get(dsl_symbol, (None, None))

    if mutate_function is None:
        print(f"Warning: No mutation function defined for DSL symbol '{dsl_symbol}'. Skipping this group.")
        continue

    for rule_input_pair in initial_rules_list:
        mutated_rules: list = mutate_function(rule_input_pair, num_variants=num_variants_per_rule)
    
        rule_input_pair["output_grid"] = execute_dsl_rule_on_grid(
                input_grid_list=rule_input_pair['input_grid'],
                dsl_rule_str=rule_input_pair['dsl_rule']
            ).tolist()
        outputed_dataset.append(rule_input_pair)
        
        for mutated_pair in mutated_rules:
            mutated_pair["output_grid"] = execute_dsl_rule_on_grid(
                input_grid_list=mutated_pair['input_grid'],
                dsl_rule_str=mutated_pair['dsl_rule']
            ).tolist()
            outputed_dataset.append(mutated_pair)

2025-07-28 00:34:41,842 - core.dsl_symbolic_executor - INFO - Executor initialization started.
2025-07-28 00:34:41,843 - core.dsl_symbolic_executor - INFO - Executor initialization complete.
2025-07-28 00:34:41,844 - core.dsl_symbolic_executor - INFO - Starting DSL program execution.
2025-07-28 00:34:41,844 - core.dsl_symbolic_executor - INFO - DSL program execution completed successfully.


TypeError: 'NoneType' object is not iterable

In [30]:
import json
from pprint import pprint
# print(json.dumps(outputed_dataset, indent=4))
pprint(outputed_dataset, width=500)

[{'dsl_rule': '⏚(∅)', 'input_grid': [[0, 0, 0], [0, 1, 0], [0, 0, 0]], 'output_grid': [[1, 1, 1], [1, 0, 1], [1, 1, 1]]},
 {'dsl_rule': '⏚(∅)', 'input_grid': [[7, 9, 7, 1, 6, 1, 3, 2, 8, 5, 2, 9, 1], [8, 6, 7, 3, 6, 6, 9, 9, 9, 5, 8, 4, 7], [1, 5, 4, 2, 4, 8, 3, 5, 2, 2, 8, 2, 4]], 'output_grid': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]},
 {'dsl_rule': '⏚(∅)', 'input_grid': [[0, 0], [0, 0]], 'output_grid': [[1, 1], [1, 1]]},
 {'dsl_rule': '⏚(∅)',
  'input_grid': [[3, 4, 8, 9, 3, 6, 7, 6, 9, 1, 5], [3, 7, 5, 5, 1, 4, 7, 4, 5, 7, 5], [1, 6, 9, 2, 1, 9, 5, 4, 6, 1, 8], [7, 7, 3, 3, 2, 9, 5, 2, 2, 6, 9], [6, 5, 2, 8, 1, 5, 6, 3, 2, 1, 8], [1, 1, 7, 7, 9, 6, 7, 4, 9, 7, 1], [2, 8, 7, 9, 5, 4, 4, 4, 1, 2, 6], [1, 5, 9, 6, 3, 5, 1, 4, 6, 4, 8], [6, 6, 2, 6, 8, 6, 7, 4, 4, 5, 6], [9, 3, 6, 1, 6, 7, 7, 4, 8, 9, 1]],
  'output_grid': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 

In [25]:
groups_list = list(groups.items())
groups_list[7]

('⏚',
 [{'input_grid': [[0, 0, 0], [0, 1, 0], [0, 0, 0]],
   'dsl_rule': '⏚(∅)',
   'output_grid': [[1, 1, 1], [1, 0, 1], [1, 1, 1]]},
  {'input_grid': [[0, 0], [0, 0]],
   'dsl_rule': '⏚(∅)',
   'output_grid': [[1, 1], [1, 1]]},
  {'input_grid': [[0, 1, 0], [1, 1, 1], [0, 1, 0]],
   'dsl_rule': '⏚(∅)',
   'output_grid': [[1, 0, 1], [0, 0, 0], [1, 0, 1]]},
  {'input_grid': [[5, 1, 5], [1, 2, 1], [5, 1, 5]],
   'dsl_rule': '⏚(V)',
   'output_grid': [[1, 0, 1], [0, 0, 0], [1, 0, 1]]},
  {'input_grid': [[0, 0, 0], [0, 7, 0], [0, 0, 0]],
   'dsl_rule': '⏚(∅)',
   'output_grid': [[1, 1, 1], [1, 0, 1], [1, 1, 1]]},
  {'input_grid': [[0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0]],
   'dsl_rule': '⏚(∅)',
   'output_grid': [[1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1]]},
  {'input_grid': [[3, 3, 3, 3, 3],
    [3, 0, 0, 0, 3],
    [3, 0, 3, 0, 3],
    [3, 0, 

In [18]:
len(groups_list) 

19

In [19]:
# groups['↕']